## Import Libraries

In [1]:
import os
os.chdir('/Users/mkesselring/Desktop/Documents/IME 541/')
import pandas as pd, numpy as np, matplotlib.pyplot as plt, networkx as nx, random
all_data = ['j120_1.csv', \
            'j120_1_2.csv', \
            'j120_1_3.csv', \
            'j120_1_4.csv', \
            'j120_1_5.csv', \
            'j120_1_6.csv', \
            'j120_1_7.csv', \
            'j120_1_8.csv', \
            'j120_1_9.csv', \
            'j120_1_10.csv', \
            'j120_2_1.csv', \
            'j120_2_2.csv', \
            'j120_2_3.csv', \
            'j120_2_4.csv', \
            'j120_2_5.csv', \
            'j120_2_6.csv', \
            'j120_2_7.csv', \
            'j120_2_8.csv', \
            'j120_2_9.csv', \
            'j120_2_10.csv' \
           ]

## Read in Instances

In [105]:
random.seed(a = 8)
data = random.sample(all_data, 10)

#resource dataframe
df = pd.read_csv(data[0], encoding='utf-8', skiprows = 3, nrows = 4)
df.columns = np.arange(1,121)        #label columns & rows
df.rename(index={0:'r1',1:'r2',2:'r3',3:'r4'}, inplace=True)
df[1] = df[1].str.strip('rr = [| ')    #data cleaning
df[120] = df[120].str.strip(' |];')
df.columns = np.arange(1,121)          #change columns and join

#duration dataframe
df_dur = pd.read_csv(data[0], delimiter = ' ,', skiprows = 2, nrows = 1, engine='python')
df_dur.columns = ['col1']        #data cleaning
df_dur['col1'] = df_dur['col1'].str.strip('d = [')
df_dur['col1'] = df_dur['col1'].str.strip('];')
df_dur.rename(index={0:'duration'}, inplace=True)     #label duration columns & rows
df_dur = df_dur.col1.str.split(",",expand=True,)
df_dur.columns = np.arange(1,121)

#successor dataframe
df_suc = pd.read_csv(data[0], skiprows = 7,header = 0, engine='python')
df_suc.columns = np.arange(1,121)
for i in np.arange(4,121):  #data cleaning
    df_suc = df_suc.drop([i],axis=1)

df_suc[1] = df_suc[1].str.strip('suc = [ { }; ]')
df_suc[2] = df_suc[2].str.strip('}')
df_suc[3] = df_suc[3].str.strip('}')
fill = 0   #filling variable
df_suc = (df_suc.T)     #change columns and join
df_suc.columns = np.arange(1,121)
df_suc = df_suc.fillna(fill)      #replace NaN, blanks, and None
df_suc = df_suc.replace('',fill)

#############################################################################################################

# chain together additional instances
x = 121

for i in data[1:] :
    cols = np.arange(x,x+120)
    
    #resources
    df1 = pd.read_csv(i, encoding='utf-8', skiprows = 3, nrows = 4)
    df1.columns = cols       #label columns & rows; data cleaning
    df1.rename(index={0:'r1',1:'r2',2:'r3',3:'r4'}, inplace=True)
    df1[x] = df1[x].str.strip('rr = [| ')
    df1[x+119] = df1[x+119].str.strip(' |];')
    df.T         #join resources
    df1.T
    df = df.join(df1)
    
    #durations
    df2 = pd.read_csv(i, delimiter = ' ,', skiprows = 2, nrows = 1, engine='python')
    df2 = df2['n_tasks = 120;'].str.split(",",expand=True,) #label duration columns & rows; data cleaning
    df2.columns = cols
    df2[x] = df2[x].str.strip('d = [')
    df2[x+119] = df2[x+119].str.strip('];')
    df2.rename(index={0:'duration'}, inplace=True)
    df2.T     #join durations
    df_dur.T
    df_dur = df_dur.join(df2)
    
    #successors
    df_suc2 = pd.read_csv(i, skiprows = 7,header = 0, engine='python')
    df_suc2.columns = np.arange(1,121)
    for j in np.arange(4,121):            #data cleaning
        df_suc2 = df_suc2.drop([j],axis=1)
    df_suc2[1] = df_suc2[1].str.strip('suc = [ { }; ]')
    df_suc2[2] = df_suc2[2].str.strip('}')
    df_suc2[3] = df_suc2[3].str.strip('}')
    fill = 0                           #filling variable
    df_suc2 = df_suc2.T                #change columns 
    df_suc2.columns = np.arange(1,121)
    df_suc2 = df_suc2.fillna(fill)     #replace NaN, blanks, and None
    df_suc2 = df_suc2.replace('',fill)
    df_suc2 = df_suc2.T.set_index([cols])
    df_suc.astype(int)
    for j in np.arange(x,x+120):        #increase job numbers inside table
        if df_suc2[1].loc[j] != 0:
            df_suc2[1].loc[j] = int(df_suc2[1].loc[j]) + x - 1
        if df_suc2[2].loc[j] != 0:
            df_suc2[2].loc[j] = int(df_suc2[2].loc[j]) + x - 1
        if df_suc2[3].loc[j] != 0:
            df_suc2[3].loc[j] = int(df_suc2[3].loc[j]) + x - 1
    df_suc = df_suc.join(df_suc2.T)     #join successors
    
    #increment for next instance
    x = x + 120   

    
#############################################################################################################

# Fix Data Frames
r = df.T
r = r.astype(int)
#r = r.iloc[1:, :]
succ = df_suc.transpose()
succ = succ.astype(int)
#succ = succ.iloc[1:,:]
d = df_dur.transpose()
d = d.astype(int)
#d = d.iloc[1:,:]

/Users/mkesselring/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


## Set Resource Constraints

In [53]:
rc = [50, 50, 50, 50]
#rc = [sum(restraints.iloc[:,0])*.5, \
#     sum(restraints.iloc[:,1])*.5, \
#     sum(restraints.iloc[:,2])*.5, \
#     sum(restraints.iloc[:,3])*.5]

## Initializations

In [110]:
# Get Precedences
pred = np.zeros((len(d), 3))
pred = pd.DataFrame(data = pred)
pred = pred.astype(int)

comp = 0
for i in np.arange(0, len(d)):
    for j in np.arange(0,3):
        job = succ.iloc[i,j]
        if job > 0:
            k = 0
            comp = 0
            while comp == 0:
                if(pred.iloc[job - 1, k] == 0):
                    pred.iloc[job - 1, k] = i + 1
                    comp = 1
                k = k + 1  

                
# Initialize Available Jobs
ava = {'j' : 0, 'd' : 0, 'r1' : 0, 'r2' : 0, 'r3' : 0, 'r4' : 0, 'rt' : 0, 's' : 0}
ava = pd.DataFrame(data = ava, index = [0])

temp = {'j' : 0, 'd' : 0, 'r1' : 0, 'r2' : 0, 'r3' : 0, 'r4' : 0, 'rt' : 0, 's' : 0}
temp = pd.DataFrame(data = temp, index = [0])

k = 1
for i in np.arange(0, len(d)):
    if (sum(pred.iloc[i, :]) == 0):
        temp['j'] = i + 1
        temp['d'] = d.iloc[i,0]
        temp['r1'] = r.iloc[i,0]
        temp['r2'] = r.iloc[i,1]
        temp['r3'] = r.iloc[i,2]
        temp['r4'] = r.iloc[i,3]
        temp['rt'] = temp['r1'] + temp['r2'] + temp['r3'] + temp['r4']
        temp['s'] = 3 - np.where(succ.iloc[i,:] == 0)[0].shape[0]
        ava = ava.append(temp)
        k = k + 1
ava = ava.iloc[1:, :]       
                
               
# Initialize Gantt Chart and Resource Profiles
gantt = np.zeros((len(d), sum(d.iloc[:,0])))
gantt = pd.DataFrame(data = gantt)
gantt = gantt.astype(int)

rec_pro = np.zeros((4, sum(d.iloc[:,0])))
rec_pro = pd.DataFrame(data = rec_pro)
rec_pro = rec_pro.astype(int)

ended = [0]
makespan = 0

output = np.zeros((len(d), 3))
output = pd.DataFrame(data  = output)
output = output.astype(int)
output.iloc[:,0] = np.arange(1,len(d) + 1)

## Mega Loop

In [ ]:
%%time
t = 0
while makespan == 0:      # while we haven't finished
    #ava = ava.sort_values('d')    # sorting heuristic   
    ava = ava.sort_values('d', ascending = False)
    #ava = ava.sort_values('rt')
    #ava = ava.sort_values('rt', ascending  = False) 
    #ava = ava.sort_values('s')
    #ava = ava.sort_values('s', ascending  = False) 
    started = [0]
    for i in np.arange(0,ava.shape[0]):       # for available jobs
        job = ava.iloc[i,0]
        rec_ava = rc - rec_pro.iloc[:,t]
        if(sum(np.subtract(rec_ava, r.iloc[job - 1,:]) - \
               abs(np.subtract(rec_ava, r.iloc[job - 1,:]))) == 0):  # if there's enough resources
            gantt.iloc[job - 1, t:(t + d.iloc[job - 1][0])] = 1     # gantt from t to t+d is 1
            rec_job = np.tile(r.iloc[job - 1,:], (d.iloc[job - 1][0], 1))    # resources for job i
            rec_pro.iloc[:, t:(t + d.iloc[job - 1][0])] = \
            np.add(rec_pro.iloc[:, t:(t + d.iloc[job - 1][0])], rec_job.T)
            started.extend([i])
            output.iloc[job - 1, 1] = t
            output.iloc[job - 1, 2] = t + d.iloc[job - 1][0]
    started = started[1:]
    mask = np.ones(ava.shape,dtype=bool)
    mask[started] = False
    ava = ava.iloc[np.where(mask[:,0] == True)[0],:]
            
    ending = np.where(gantt.iloc[:,t] - gantt.iloc[:,t+1] == 1)[0]   # find jobs that are ending in this period
    if len(ending) > 0:
        ending = ending + 1
        ended = np.append(ended, ending)      # add to already ended jobs
        for j in ending:  # for ending jobs
            ending_pred = np.where(pred == j)
            pred.iloc[ending_pred] = 0    # find what jobs have an ending predecessor
            for i in ending_pred[0]:
                if(sum(pred.iloc[i, :]) == 0):    # add any jobs with no predecessors to ava
                    temp['j'] = i + 1
                    temp['d'] = d.iloc[i,0]
                    temp['r1'] = r.iloc[i,0]
                    temp['r2'] = r.iloc[i,1]
                    temp['r3'] = r.iloc[i,2]
                    temp['r4'] = r.iloc[i,3]
                    temp['rt'] = temp['r1'] + temp['r2'] + temp['r3'] + temp['r4']
                    temp['s'] = 3 - np.where(succ.iloc[i,:] == 0)[0].shape[0]
                    ava = ava.append(temp)           
    
    if(len(ended) == len(d)):          # check if we finished all the jobs
        makespan = t
    #gantt.join([np.zeros((len(d), 1))])
    #rec_pro.join([np.zeros((len(d), 1))])
#    makespan = 1
    t = t + 1
makespan

In [109]:
output.to_csv("/Users/mkesselring/Desktop/Documents/IME 541/1200_sol_8.1.csv")

In [108]:
output.max()

0    1200
1     231
2     235
dtype: int64